In [1]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import numpy as np
from pprint import pprint as pp
import csv
from pathlib import Path
import seaborn as sns
from itertools import product
import string

import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import BorderlineSMOTE
from imblearn.pipeline import Pipeline 

from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import r2_score, classification_report, confusion_matrix, accuracy_score, roc_auc_score, roc_curve, precision_recall_curve, average_precision_score
from sklearn.metrics import homogeneity_score, silhouette_score
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import MiniBatchKMeans, DBSCAN
import fasttext

import gensim
from gensim import corpora

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, precision_score, recall_score, f1_score, accuracy_score, confusion_matrix

In [4]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, precision_score, recall_score, f1_score, accuracy_score, confusion_matrix
from sklearn.model_selection import cross_val_predict

In [5]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTEENN
from imblearn.combine import SMOTETomek

In [6]:
pd.set_option('display.max_columns', None)

### Retrieving the data

#### DF 2

In [7]:
df2 = pd.read_csv('df5.csv')

In [8]:
df2.head(2)

,Unnamed: 0,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens,target
0,0,1077501.0,1296599.0,5000.0,5000.0,4975.0,0,0.1065,162.87,1,6,25275,10.0,4,24000.0,2,2011.0,0,1,3160,860,3,27.65,0.0,1985.0,735.0,739.0,1.0,500.0,222.0,3.0,0.0,13648.0,0.837,9.0,0.0,0.0,5863.155187,5833.84,5000.00,863.16,0.0,0.00,0.00,2015.0,171.62,0.0,2016.0,744.0,740.0,0,0,0,0.0,0.0,0,1
1,1,1077430.0,1314167.0,2500.0,2500.0,2500.0,1,0.1527,59.83,2,13,20173,1.0,4,30000.0,1,2011.0,0,0,17558,309,10,1.00,0.0,1999.0,740.0,744.0,5.0,500.0,222.0,3.0,0.0,1687.0,0.094,4.0,0.0,0.0,1008.710000,1008.71,456.46,435.17,0.0,117.08,1.11,2013.0,119.66,0.0,2016.0,499.0,0.0,0,0,0,0.0,0.0,0,2


# Models

In [9]:
# Initialize lists to store evaluation metrics for each fold
dataset_used = []
model_used = []
data_balancing_technique = []
accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []
conf_matrices = []

In [10]:
#combined_metrics = pd.DataFrame()

combined_metrics = pd.DataFrame(columns=['dataset', 'model', 'data balancing technique', 'fold', 'precision_1','precision_2','recall_1','recall_2','f1-score_1','f1-score_2','support_1','support_2','TP','FP','TN','FN'])

# Fast Text 
### train_supervised

In [ ]:
# Define hyperparameters
dim = 300  # Dimension of word vectors
min_count = 1  # Minimum frequency of words
loss = 'ns'  # Hierarchical softmax loss #'hs', 'ns', 'softmax'
epoch = 100  # Number of training epochs
bucket = 2000000  # Number of buckets used for hashing n-grams
word_ngrams = 1  # Maximum length of word n-grams
lr = 0.5 #0.1, 0.01, 0.001 #learning rate

## DF2

In [12]:
# Assuming 'df2' is your DataFrame with features and the target column
# Replace 'features' with the actual list of feature columns
features = df2.drop(columns=['target'])  # Drop the target column to get the feature columns
target = '__label__' + df2['target'].astype(str)

# Initialize KFold with 10 folds
kf = KFold(n_splits=10, shuffle=True, random_state=42)

i = 1

# Perform k-fold cross-validation
for train_index, test_index in kf.split(features):
    X_train, X_test = features.iloc[train_index], features.iloc[test_index]
    y_train, y_test = target.iloc[train_index], target.iloc[test_index]
    
    # Create a new column "content" by concatenating values from selected columns
    X_train['content'] = y_train + ' ' + X_train.apply(lambda row: ' '.join(str(val) for val in row), axis=1)
    X_test['content'] = X_test.apply(lambda row: ' '.join(str(val) for val in row), axis=1)
    
    X_train = X_train['content']
    X_test = X_test['content']
    
    X_train_list = [value for value in X_train]
    
    # Save training data to a file
    with open("train_german_fast_text.txt", "w") as f:
        for line in X_train_list:
            f.write(line + "\n")

    # Training data file path
    train_data_path = "train_german_fast_text.txt"
    
    # Train the supervised model
    model = fasttext.train_supervised(input=train_data_path, dim=dim, minCount=min_count,bucket=bucket, loss=loss, epoch=epoch, wordNgrams=word_ngrams)#lr=lr, 
                                       #lr=lr,bucket=bucket,
    
    # Predict on the testing set
    # Initialize an empty list to store the predicted labels
    y_pred_list = []

    # Iterate over each data point in X_test and predict the label
    for text in X_test:
        # Predict the label for the current text
        predicted_label, _ = model.predict(text)
        # Append the predicted label to the list
        y_pred_list.append(predicted_label[0])

    # Convert the list of predicted labels to a pandas Series
    y_pred = pd.Series(y_pred_list)
    
    #y_pred = np.round(y_pred)
    
    #y_pred[y_pred <= 0] = 1
    #y_pred[y_pred >= 2] = 2
    
    # Extract the numeric label using a lambda function
    y_test = y_test.apply(lambda x: int(x.split("__label__")[1]))  ## uncomment for next run
    y_pred = y_pred.apply(lambda x: int(x.split("__label__")[1]))
    
    # Model evaluation
    accuracy = accuracy_score(y_test, y_pred.round())
    precision = precision_score(y_test, y_pred.round(), average='weighted')
    recall = recall_score(y_test, y_pred.round(), average='weighted')
    f1 = f1_score(y_test, y_pred.round(), average='weighted')
    conf_matrix = confusion_matrix(y_test, y_pred.round())
    
    
    # Convert classification report to DataFrame
    report_df = pd.DataFrame(classification_report(y_test, y_pred.round(), output_dict=True)).transpose()
    # Convert confusion matrix to DataFrame
    matrix_df = pd.DataFrame(confusion_matrix(y_test, y_pred.round()))
    
    # Extract metrics for class 1
    metrics_1 = report_df.loc['1', ['precision', 'recall', 'f1-score', 'support']]

    # Extract metrics for class 2
    metrics_2 = report_df.loc['2', ['precision', 'recall', 'f1-score', 'support']]

    # Extract TP, TN, FP, FN counts from the confusion matrix DataFrame
    TP = matrix_df.loc[0, 0]
    TN = matrix_df.loc[1, 1]
    FP = matrix_df.loc[1, 0]
    FN = matrix_df.loc[0, 1]
    
    new_metric_row = {
    'dataset': 'DF2',
    'model' : 'Fast Text - train_supervised',
    'data balancing technique' : 'None',
    'fold' : i,
    'precision_1': metrics_1['precision'],
    'precision_2': metrics_2['precision'],
    'recall_1': metrics_1['recall'],
    'recall_2': metrics_2['recall'],
    'f1-score_1': metrics_1['f1-score'],
    'f1-score_2': metrics_2['f1-score'],
    'support_1': metrics_1['support'],
    'support_2': metrics_2['support'],
    'TP' : TP,
    'FP' : FP,
    'TN' : TN,
    'FN' : FN
    }
    
    i = i + 1
    
    #combined_metrics = combined_metrics.append(new_metric_row, ignore_index=True)
    combined_metrics.loc[len(combined_metrics)] = new_metric_row
    
    
    # Append evaluation metrics to lists
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    f1_scores.append(f1)
    conf_matrices.append(conf_matrix)

# Calculate mean evaluation metrics across all folds
mean_accuracy = np.mean(accuracy_scores) #sum(accuracy_scores) / len(accuracy_scores)
mean_precision = np.mean(precision_scores) #sum(precision_scores) / len(precision_scores)
mean_recall = np.mean(recall_scores) #sum(recall_scores) / len(recall_scores)
mean_f1 = np.mean(f1_scores) #sum(f1_scores) / len(f1_scores)

print('Mean Accuracy: {:.2f}'.format(mean_accuracy))
print('Mean Precision: {:.2f}'.format(mean_precision))
print('Mean Recall: {:.2f}'.format(mean_recall))
print('Mean F1-Score: {:.2f}'.format(mean_f1))

Mean Accuracy: 0.92
Mean Precision: 0.92
Mean Recall: 0.92
Mean F1-Score: 0.92


In [15]:
combined_metrics[(combined_metrics['data balancing technique'] == 'None') & (combined_metrics['dataset'] == 'DF2')]#.head(2)

,dataset,model,data balancing technique,fold,precision_1,precision_2,recall_1,recall_2,f1-score_1,f1-score_2,support_1,support_2,TP,FP,TN,FN
0,DF2,Fast Text - train_supervised,None,1,0.940644,0.780684,0.970080,0.635025,0.955135,0.700361,3643.0,611.0,3534,223,388,109
1,DF2,Fast Text - train_supervised,None,2,0.941240,0.764706,0.965859,0.647910,0.953390,0.701480,3632.0,622.0,3508,219,403,124
2,DF2,Fast Text - train_supervised,None,3,0.936682,0.788650,0.970116,0.629687,0.953106,0.700261,3614.0,640.0,3506,237,403,108
3,DF2,Fast Text - train_supervised,None,4,0.938341,0.768519,0.965374,0.644410,0.951666,0.701014,3610.0,644.0,3485,229,415,125
4,DF2,Fast Text - train_supervised,None,5,0.943019,0.780399,0.966510,0.670827,0.954620,0.721477,3613.0,641.0,3492,211,430,121
5,DF2,Fast Text - train_supervised,None,6,0.946943,0.820370,0.973153,0.692187,0.959869,0.750847,3613.0,640.0,3516,197,443,97
6,DF2,Fast Text - train_supervised,None,7,0.939925,0.807763,0.971055,0.662121,0.955236,0.727727,3593.0,660.0,3489,223,437,104
7,DF2,Fast Text - train_supervised,None,8,0.938255,0.833333,0.975440,0.656716,0.956486,0.734558,3583.0,670.0,3495,230,440,88
8,DF2,Fast Text - train_supervised,None,9,0.944037,0.797203,0.967697,0.688822,0.955721,0.739060,3591.0,662.0,3475,206,456,116
9,DF2,Fast Text - train_supervised,None,10,0.931419,0.782077,0.970368,0.598131,0.950495,0.677846,3611.0,642.0,3504,258,384,107


### FastText train_supervised with df2 data and imbalance data tackling (RandomUnderSampler)

In [16]:
# Assuming 'df2' is your DataFrame with features and the target column
# Replace 'features' with the actual list of feature columns
features = df2.drop(columns=['target'])  # Drop the target column to get the feature columns
target = '__label__' + df2['target'].astype(str)

# Initialize KFold with 10 folds
kf = KFold(n_splits=10, shuffle=True, random_state=42)

i = 1

# Perform k-fold cross-validation
for train_index, test_index in kf.split(features):
    X_train, X_test = features.iloc[train_index], features.iloc[test_index]
    y_train, y_test = target.iloc[train_index], target.iloc[test_index]
    
    # Oversampling the minority class using SMOTE
    rus = RandomUnderSampler(random_state=42)
    X_train_resampled, y_train_resampled = rus.fit_resample(X_train, y_train)
    
    # Create a new column "content" by concatenating values from selected columns
    X_train_resampled['content'] = y_train_resampled + ' ' + X_train_resampled.apply(lambda row: ' '.join(str(val) for val in row), axis=1)
    X_test['content'] = X_test.apply(lambda row: ' '.join(str(val) for val in row), axis=1)
    
    X_train = X_train_resampled['content']
    X_test = X_test['content']
    
    X_train_list = [value for value in X_train]
    
    # Save training data to a file
    with open("train_german_fast_text.txt", "w") as f:
        for line in X_train_list:
            f.write(line + "\n")

    # Training data file path
    train_data_path = "train_german_fast_text.txt"
    
    # Train the supervised model
    model = fasttext.train_supervised(input=train_data_path, dim=dim, minCount=min_count,bucket=bucket, loss=loss, epoch=epoch, wordNgrams=word_ngrams)#lr=lr, 
                                       #lr=lr,bucket=bucket,
    
    # Predict on the testing set
    # Initialize an empty list to store the predicted labels
    y_pred_list = []

    # Iterate over each data point in X_test and predict the label
    for text in X_test:
        # Predict the label for the current text
        predicted_label, _ = model.predict(text)
        # Append the predicted label to the list
        y_pred_list.append(predicted_label[0])

    # Convert the list of predicted labels to a pandas Series
    y_pred = pd.Series(y_pred_list)
    
    # Extract the numeric label using a lambda function
    y_test = y_test.apply(lambda x: int(x.split("__label__")[1]))  ## uncomment for next run
    y_pred = y_pred.apply(lambda x: int(x.split("__label__")[1]))
    
    # Model evaluation
    accuracy = accuracy_score(y_test, y_pred.round())
    precision = precision_score(y_test, y_pred.round(), average='weighted')
    recall = recall_score(y_test, y_pred.round(), average='weighted')
    f1 = f1_score(y_test, y_pred.round(), average='weighted')
    conf_matrix = confusion_matrix(y_test, y_pred.round())
    
    
    # Convert classification report to DataFrame
    report_df = pd.DataFrame(classification_report(y_test, y_pred.round(), output_dict=True)).transpose()
    # Convert confusion matrix to DataFrame
    matrix_df = pd.DataFrame(confusion_matrix(y_test, y_pred.round()))
    
    # Extract metrics for class 1
    metrics_1 = report_df.loc['1', ['precision', 'recall', 'f1-score', 'support']]

    # Extract metrics for class 2
    metrics_2 = report_df.loc['2', ['precision', 'recall', 'f1-score', 'support']]

    # Extract TP, TN, FP, FN counts from the confusion matrix DataFrame
    TP = matrix_df.loc[0, 0]
    TN = matrix_df.loc[1, 1]
    FP = matrix_df.loc[1, 0]
    FN = matrix_df.loc[0, 1]
    
    new_metric_row = {
    'dataset': 'DF2',
    'model' : 'Fast Text - train_supervised',
    'data balancing technique' : 'Random Under Sampler',
    'fold' : i,
    'precision_1': metrics_1['precision'],
    'precision_2': metrics_2['precision'],
    'recall_1': metrics_1['recall'],
    'recall_2': metrics_2['recall'],
    'f1-score_1': metrics_1['f1-score'],
    'f1-score_2': metrics_2['f1-score'],
    'support_1': metrics_1['support'],
    'support_2': metrics_2['support'],
    'TP' : TP,
    'FP' : FP,
    'TN' : TN,
    'FN' : FN
    }
    
    i = i + 1
    
    #combined_metrics = combined_metrics.append(new_metric_row, ignore_index=True)
    combined_metrics.loc[len(combined_metrics)] = new_metric_row
    
    
    # Append evaluation metrics to lists
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    f1_scores.append(f1)
    conf_matrices.append(conf_matrix)

# Calculate mean evaluation metrics across all folds
mean_accuracy = np.mean(accuracy_scores) #sum(accuracy_scores) / len(accuracy_scores)
mean_precision = np.mean(precision_scores) #sum(precision_scores) / len(precision_scores)
mean_recall = np.mean(recall_scores) #sum(recall_scores) / len(recall_scores)
mean_f1 = np.mean(f1_scores) #sum(f1_scores) / len(f1_scores)

print('Mean Accuracy: {:.2f}'.format(mean_accuracy))
print('Mean Precision: {:.2f}'.format(mean_precision))
print('Mean Recall: {:.2f}'.format(mean_recall))
print('Mean F1-Score: {:.2f}'.format(mean_f1))

Mean Accuracy: 0.90
Mean Precision: 0.91
Mean Recall: 0.90
Mean F1-Score: 0.90


In [18]:
combined_metrics[(combined_metrics['data balancing technique'] == 'Random Under Sampler') & (combined_metrics['dataset'] == 'DF2')]#.head(2)

,dataset,model,data balancing technique,fold,precision_1,precision_2,recall_1,recall_2,f1-score_1,f1-score_2,support_1,support_2,TP,FP,TN,FN
10,DF2,Fast Text - train_supervised,Random Under Sampler,1,0.977832,0.559834,0.883887,0.880524,0.928489,0.684478,3643.0,611.0,3220,73,538,423
11,DF2,Fast Text - train_supervised,Random Under Sampler,2,0.982573,0.516393,0.853800,0.911576,0.913671,0.659302,3632.0,622.0,3101,55,567,531
12,DF2,Fast Text - train_supervised,Random Under Sampler,3,0.975248,0.547945,0.872164,0.875000,0.920830,0.673887,3614.0,640.0,3152,80,560,462
13,DF2,Fast Text - train_supervised,Random Under Sampler,4,0.978192,0.527523,0.857341,0.892857,0.913788,0.663206,3610.0,644.0,3095,69,575,515
14,DF2,Fast Text - train_supervised,Random Under Sampler,5,0.977123,0.534337,0.862995,0.886115,0.916520,0.666667,3613.0,641.0,3118,73,568,495
15,DF2,Fast Text - train_supervised,Random Under Sampler,6,0.978158,0.521938,0.855245,0.892188,0.912581,0.658593,3613.0,640.0,3090,69,571,523
16,DF2,Fast Text - train_supervised,Random Under Sampler,7,0.973864,0.554379,0.871138,0.872727,0.919642,0.678046,3593.0,660.0,3130,84,576,463
17,DF2,Fast Text - train_supervised,Random Under Sampler,8,0.976577,0.566127,0.872732,0.888060,0.921739,0.691458,3583.0,670.0,3127,75,595,456
18,DF2,Fast Text - train_supervised,Random Under Sampler,9,0.983943,0.537313,0.853244,0.924471,0.913945,0.679622,3591.0,662.0,3064,50,612,527
19,DF2,Fast Text - train_supervised,Random Under Sampler,10,0.973814,0.553128,0.875381,0.867601,0.921978,0.675561,3611.0,642.0,3161,85,557,450


### FastText train_supervised with df2 data and imbalance data tackling (RandomOverSampler)

In [19]:
# Assuming 'df2' is your DataFrame with features and the target column
# Replace 'features' with the actual list of feature columns
features = df2.drop(columns=['target'])  # Drop the target column to get the feature columns
target = '__label__' + df2['target'].astype(str)

# Initialize KFold with 10 folds
kf = KFold(n_splits=10, shuffle=True, random_state=42)

i = 1

# Perform k-fold cross-validation
for train_index, test_index in kf.split(features):
    X_train, X_test = features.iloc[train_index], features.iloc[test_index]
    y_train, y_test = target.iloc[train_index], target.iloc[test_index]
    
    # Oversampling the minority class using SMOTE
    ros = RandomOverSampler(random_state=42)
    X_train_resampled, y_train_resampled = ros.fit_resample(X_train, y_train)
    
    # Create a new column "content" by concatenating values from selected columns
    X_train_resampled['content'] = y_train_resampled + ' ' + X_train_resampled.apply(lambda row: ' '.join(str(val) for val in row), axis=1)
    X_test['content'] = X_test.apply(lambda row: ' '.join(str(val) for val in row), axis=1)
    
    X_train = X_train_resampled['content']
    X_test = X_test['content']
    
    X_train_list = [value for value in X_train]
    
    # Save training data to a file
    with open("train_german_fast_text.txt", "w") as f:
        for line in X_train_list:
            f.write(line + "\n")

    # Training data file path
    train_data_path = "train_german_fast_text.txt"
    
    # Train the supervised model
    model = fasttext.train_supervised(input=train_data_path, dim=dim, minCount=min_count,bucket=bucket, loss=loss, epoch=epoch, wordNgrams=word_ngrams)#lr=lr, 
                                       #lr=lr,bucket=bucket,
    
    # Predict on the testing set
    # Initialize an empty list to store the predicted labels
    y_pred_list = []

    # Iterate over each data point in X_test and predict the label
    for text in X_test:
        # Predict the label for the current text
        predicted_label, _ = model.predict(text)
        # Append the predicted label to the list
        y_pred_list.append(predicted_label[0])

    # Convert the list of predicted labels to a pandas Series
    y_pred = pd.Series(y_pred_list)
    
    # Extract the numeric label using a lambda function
    y_test = y_test.apply(lambda x: int(x.split("__label__")[1]))  ## uncomment for next run
    y_pred = y_pred.apply(lambda x: int(x.split("__label__")[1]))
    
    # Model evaluation
    accuracy = accuracy_score(y_test, y_pred.round())
    precision = precision_score(y_test, y_pred.round(), average='weighted')
    recall = recall_score(y_test, y_pred.round(), average='weighted')
    f1 = f1_score(y_test, y_pred.round(), average='weighted')
    conf_matrix = confusion_matrix(y_test, y_pred.round())
    
    
    # Convert classification report to DataFrame
    report_df = pd.DataFrame(classification_report(y_test, y_pred.round(), output_dict=True)).transpose()
    # Convert confusion matrix to DataFrame
    matrix_df = pd.DataFrame(confusion_matrix(y_test, y_pred.round()))
    
    # Extract metrics for class 1
    metrics_1 = report_df.loc['1', ['precision', 'recall', 'f1-score', 'support']]

    # Extract metrics for class 2
    metrics_2 = report_df.loc['2', ['precision', 'recall', 'f1-score', 'support']]

    # Extract TP, TN, FP, FN counts from the confusion matrix DataFrame
    TP = matrix_df.loc[0, 0]
    TN = matrix_df.loc[1, 1]
    FP = matrix_df.loc[1, 0]
    FN = matrix_df.loc[0, 1]
    
    new_metric_row = {
    'dataset': 'DF2',
    'model' : 'Fast Text - train_supervised',
    'data balancing technique' : 'Random Over Sampler',
    'fold' : i,
    'precision_1': metrics_1['precision'],
    'precision_2': metrics_2['precision'],
    'recall_1': metrics_1['recall'],
    'recall_2': metrics_2['recall'],
    'f1-score_1': metrics_1['f1-score'],
    'f1-score_2': metrics_2['f1-score'],
    'support_1': metrics_1['support'],
    'support_2': metrics_2['support'],
    'TP' : TP,
    'FP' : FP,
    'TN' : TN,
    'FN' : FN
    }
    
    i = i + 1
    
    #combined_metrics = combined_metrics.append(new_metric_row, ignore_index=True)
    combined_metrics.loc[len(combined_metrics)] = new_metric_row
    
    
    # Append evaluation metrics to lists
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    f1_scores.append(f1)
    conf_matrices.append(conf_matrix)

# Calculate mean evaluation metrics across all folds
mean_accuracy = np.mean(accuracy_scores) #sum(accuracy_scores) / len(accuracy_scores)
mean_precision = np.mean(precision_scores) #sum(precision_scores) / len(precision_scores)
mean_recall = np.mean(recall_scores) #sum(recall_scores) / len(recall_scores)
mean_f1 = np.mean(f1_scores) #sum(f1_scores) / len(f1_scores)

print('Mean Accuracy: {:.2f}'.format(mean_accuracy))
print('Mean Precision: {:.2f}'.format(mean_precision))
print('Mean Recall: {:.2f}'.format(mean_recall))
print('Mean F1-Score: {:.2f}'.format(mean_f1))

Mean Accuracy: 0.90
Mean Precision: 0.92
Mean Recall: 0.90
Mean F1-Score: 0.91


In [20]:
combined_metrics[(combined_metrics['data balancing technique'] == 'Random Over Sampler') & (combined_metrics['dataset'] == 'DF2')]#.head(2)

,dataset,model,data balancing technique,fold,precision_1,precision_2,recall_1,recall_2,f1-score_1,f1-score_2,support_1,support_2,TP,FP,TN,FN
20,DF2,Fast Text - train_supervised,Random Over Sampler,1,0.941349,0.777336,0.969256,0.639935,0.955099,0.701975,3643.0,611.0,3531,220,391,112
21,DF2,Fast Text - train_supervised,Random Over Sampler,2,0.941555,0.770992,0.966960,0.649518,0.954089,0.705061,3632.0,622.0,3512,218,404,120
22,DF2,Fast Text - train_supervised,Random Over Sampler,3,0.939930,0.792381,0.969840,0.650000,0.954651,0.714163,3614.0,640.0,3505,224,416,109
23,DF2,Fast Text - train_supervised,Random Over Sampler,4,0.943811,0.766667,0.963158,0.678571,0.953386,0.719934,3610.0,644.0,3477,207,437,133
24,DF2,Fast Text - train_supervised,Random Over Sampler,5,0.943452,0.774194,0.965126,0.673947,0.954166,0.720601,3613.0,641.0,3487,209,432,126
25,DF2,Fast Text - train_supervised,Random Over Sampler,6,0.946602,0.811009,0.971492,0.690625,0.958885,0.745992,3613.0,640.0,3510,198,442,103
26,DF2,Fast Text - train_supervised,Random Over Sampler,7,0.945139,0.802102,0.968550,0.693939,0.956701,0.744110,3593.0,660.0,3480,202,458,113
27,DF2,Fast Text - train_supervised,Random Over Sampler,8,0.940399,0.823853,0.973207,0.670149,0.956522,0.739095,3583.0,670.0,3487,221,449,96
28,DF2,Fast Text - train_supervised,Random Over Sampler,9,0.941176,0.789007,0.966862,0.672205,0.953846,0.725938,3591.0,662.0,3472,217,445,119
29,DF2,Fast Text - train_supervised,Random Over Sampler,10,0.932997,0.792683,0.971753,0.607477,0.951980,0.687831,3611.0,642.0,3509,252,390,102


## Performance metrics for the Linear Regression

In [22]:
combined_metrics

,dataset,model,data balancing technique,fold,precision_1,precision_2,recall_1,recall_2,f1-score_1,f1-score_2,support_1,support_2,TP,FP,TN,FN
0,DF2,Fast Text - train_supervised,None,1,0.940644,0.780684,0.970080,0.635025,0.955135,0.700361,3643.0,611.0,3534,223,388,109
1,DF2,Fast Text - train_supervised,None,2,0.941240,0.764706,0.965859,0.647910,0.953390,0.701480,3632.0,622.0,3508,219,403,124
2,DF2,Fast Text - train_supervised,None,3,0.936682,0.788650,0.970116,0.629687,0.953106,0.700261,3614.0,640.0,3506,237,403,108
3,DF2,Fast Text - train_supervised,None,4,0.938341,0.768519,0.965374,0.644410,0.951666,0.701014,3610.0,644.0,3485,229,415,125
4,DF2,Fast Text - train_supervised,None,5,0.943019,0.780399,0.966510,0.670827,0.954620,0.721477,3613.0,641.0,3492,211,430,121
5,DF2,Fast Text - train_supervised,None,6,0.946943,0.820370,0.973153,0.692187,0.959869,0.750847,3613.0,640.0,3516,197,443,97
6,DF2,Fast Text - train_supervised,None,7,0.939925,0.807763,0.971055,0.662121,0.955236,0.727727,3593.0,660.0,3489,223,437,104
7,DF2,Fast Text - train_supervised,None,8,0.938255,0.833333,0.975440,0.656716,0.956486,0.734558,3583.0,670.0,3495,230,440,88
8,DF2,Fast Text - train_supervised,None,9,0.944037,0.797203,0.967697,0.688822,0.955721,0.739060,3591.0,662.0,3475,206,456,116
9,DF2,Fast Text - train_supervised,None,10,0.931419,0.782077,0.970368,0.598131,0.950495,0.677846,3611.0,642.0,3504,258,384,107


In [23]:
# Assuming df is your DataFrame
combined_metrics.to_csv('Output Data/Fast Text - train_supervised.csv')